In [2]:
# imports necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,GridSearchCV 
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import featuretools as ft

In [10]:
# loads dataset
df = pd.read_csv("../data/challenge/train.csv")
df2 = pd.read_csv("../data/challenge/additive.csv")

#adds columns
df["cap"] = df2["CAPTC_DIA"]
df["resg"] = df2["RESG_DIA"]
df.drop(["Fluxo"],inplace=True,axis=1)

df3 = pd.read_csv("../data/challenge/CBOEVolatilityIndexHistoricalData.csv",sep=";")

date = {
    "jan": "01",
    "feb": "02",       
    "mar": "03",
    "apr": "04",
    "may": "05",
    "jun": "06",
    "ju": "06",
    "jul": "07",
    "aug": "08",
    "sep": "09",
    "oct": "10",
    "nov": "11",
    "dec": "12",
}
 
df3["Date"] = df3["Date"].apply(lambda x : x.replace(x[0:4],date[x[0:3].lower()]+"/"))
df3["Date"] = df3["Date"].apply(lambda x : x.replace(x[0:5],x[3:5]+"/"+x[0:2]))
df3["Date"] =  pd.to_datetime(df3["Date"])
df3 = df3[["Date","Price"]]

df = df.merge(df3, left_on="DT_COMPTC",right_on="Date",)
df = df.drop(["Date"], axis=1)
df = df.drop(["DT_COMPTC"], axis=1)

df["NDX_SPX"] = df["NDX"]/df['SPX']
df["Dol_NDX"] = df["Dol"]/df["NDX"]
df["IBOV_NDX"] = df["IBOV"]/df["NDX"]
df["Dol_SPX"] = df["Dol"]/df["SPX"]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_5548\205460246.py:30: UserWarning: Parsing '19/05/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df3["Date"] =  pd.to_datetime(df3["Date"])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5548\205460246.py:30: UserWarning: Parsing '18/05/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df3["Date"] =  pd.to_datetime(df3["Date"])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5548\205460246.py:30: UserWarning: Parsing '17/05/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df3["Date"] =  pd.to_datetime(df3["Date"])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5548\205460246.py:30: UserWarning: Parsing '14/05/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  df3["Date"] =  pd.to_datetime(df3["Date"])
C:\Users\Usuario

,Date,Price
0,2021-05-19,22.18
1,2021-05-18,21.34
2,2021-05-17,19.72
3,2021-05-14,18.81
4,2021-05-13,23.13
...,...,...
4084,2005-07-03,12.26
4085,2005-04-03,11.94
4086,2005-03-03,12.93
4087,2005-02-03,12.50


In [45]:
for coluna in ["Dol","IBOV","NDX","SPX"]:
    df[f"{coluna}_mean"] =df[coluna].shift(1)

for coluna in ["Dol_mean","IBOV_mean","NDX_mean","SPX_mean"]:
    df[coluna].iloc[0] =   df[coluna].iloc[1] 

In [46]:
df

,IBOV,Dol,NDX,SPX,cap,resg,Price,NDX_SPX,Dol_NDX,IBOV_NDX,Dol_SPX,Dol_mean,IBOV_mean,NDX_mean,SPX_mean
0,27730.0,26.275,1527.25,1210.41,27.239,26.758,12.04,1.261763,0.017204,18.156818,0.021708,26.275,27730.0,1527.25,1210.41
1,28199.7,2.639,1525.28,1210.08,32.365,61.563,12.50,1.260479,0.001730,18.488212,0.002181,26.275,27730.0,1527.25,1210.41
2,28668.4,2.681,1511.89,1210.47,113.145,73.632,12.93,1.249011,0.001773,18.961962,0.002215,2.639,28199.7,1525.28,1210.08
3,29197.2,2.656,1520.58,1222.13,35.034,7.341,11.94,1.244205,0.001747,19.201357,0.002173,2.681,28668.4,1511.89,1210.47
4,29455.4,26.823,1545.20,1225.31,20.495,8.256,12.26,1.261069,0.017359,19.062516,0.021891,2.656,29197.2,1520.58,1222.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,120705.9,53.087,13109.15,4112.50,1032.190,779.204,23.13,3.187635,0.004050,9.207759,0.012909,5.307,119710.0,13001.63,4063.04
3978,121880.8,52.728,13393.12,4173.85,1002.087,715.739,18.81,3.208817,0.003937,9.100254,0.012633,53.087,120705.9,13109.15,4112.50
3979,122937.9,52.751,13312.91,4163.29,887.584,1014.797,19.72,3.197690,0.003962,9.234487,0.012671,52.728,121880.8,13393.12,4173.85
3980,122980.0,52.613,13217.68,4127.83,981.088,603.063,21.34,3.202089,0.003981,9.304205,0.012746,52.751,122937.9,13312.91,4163.29


In [47]:
# padroniza os dados
scaler = MinMaxScaler()


In [48]:
df.corr().to_csv("rel.csv")